# ChatRGK developped by Khalid Regui 

here is my website : https://regui.online

In [11]:
from sk import my_sk
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=my_sk, model="gpt-4o")

In [12]:
from langchain.prompts import ChatPromptTemplate

template = """
You are a Moroccan AI fleuent in Darija.
Use the provided context to response to the user Question.
Make your response short and helpful.
If you don't find the relevent answer in the context provided just response you don't know.

Context: {context}

Question: {question}

Helpful answer in Darija : 
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
DATA_PATH = "data/"
loader = DirectoryLoader(DATA_PATH, glob='*.pdf', loader_cls = PyPDFLoader)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts=text_splitter.split_documents(documents)

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(openai_api_key=my_sk)
vdb = FAISS.from_documents(texts, embeddings)

In [ ]:
DB_FAISS_PATH = "vectorstores/db_faiss"
vdb.save_local(DB_FAISS_PATH)

In [13]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(openai_api_key=my_sk)
DB_FAISS_PATH = "vectorstores/db_faiss"
vdb = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization = True)

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [15]:
from langchain_core.runnables import RunnablePassthrough
chain = (
    {"context": vdb.as_retriever(search_kwargs={'k': 3}), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

In [ ]:
from flask import Flask, jsonify, render_template, request
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route("/get")
def response():
    query = request.args.get('msg')
    res = chain.invoke(query)
    return jsonify({"response": res})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.72.125.39:5000
Press CTRL+C to quit
10.72.125.39 - - [20/Jul/2024 23:35:49] "GET / HTTP/1.1" 200 -
10.72.125.39 - - [20/Jul/2024 23:37:58] "GET /get?msg=salam HTTP/1.1" 200 -
10.72.125.39 - - [20/Jul/2024 23:38:12] "GET / HTTP/1.1" 200 -
10.72.125.39 - - [20/Jul/2024 23:38:19] "GET /get?msg=slm HTTP/1.1" 200 -
10.72.125.39 - - [20/Jul/2024 23:38:39] "GET / HTTP/1.1" 200 -
10.72.125.39 - - [20/Jul/2024 23:38:59] "GET /get?msg=salam%20chatrgk HTTP/1.1" 200 -
10.72.125.39 - - [20/Jul/2024 23:39:16] "GET /get?msg=bghit%20des%20info%203la%20emi HTTP/1.1" 200 -
10.72.125.39 - - [20/Jul/2024 23:39:45] "GET / HTTP/1.1" 200 -
10.72.125.39 - - [20/Jul/2024 23:40:31] "GET /get?msg=Salam%20ChatRGK HTTP/1.1" 200 -
10.72.125.39 - - [20/Jul/2024 23:40:50] "GET /get?msg=bghit%20n3rf%20chno%20hya%20emi HTTP/1.1" 200 -
10.72.125.39 - - [20/Jul/2024 23:41:14] "GET /get?msg=ah%20mzn HTTP/1.1" 200 -
10.72.125.